## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/meganmcmahon/Desktop/Bootcamp/WeatherPy Mod 6/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,70.32,30,20,10.36,few clouds
1,1,Dikson,RU,73.5069,80.5464,26.44,94,16,6.40,few clouds
2,2,New Norfolk,AU,-42.7826,147.0587,42.91,83,80,1.88,light rain
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,28.40,80,100,10.36,overcast clouds
4,4,Kapoeta,SS,4.7717,33.5903,81.36,37,20,10.98,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Kapoeta,SS,4.7717,33.5903,81.36,37,20,10.98,few clouds
7,7,Quisqueya,DO,18.5554,-69.4081,89.49,53,53,12.84,broken clouds
8,8,Harper,LR,4.3750,-7.7169,79.02,82,100,5.08,overcast clouds
12,12,Tuban,ID,-6.8976,112.0649,78.03,86,100,5.21,overcast clouds
17,17,Kapaa,US,22.0752,-159.3190,76.98,85,20,8.05,few clouds
19,19,Bethel,US,41.3712,-73.4140,83.28,40,0,10.36,clear sky
20,20,Rikitea,PF,-23.1203,-134.9692,75.69,74,99,10.98,overcast clouds
24,24,Fort Dodge,US,42.4975,-94.1680,75.02,60,40,12.66,scattered clouds
25,25,Kuah,MY,6.3265,99.8432,80.58,83,20,1.14,very heavy rain
36,36,Pedernales,DO,18.0384,-71.7440,87.53,62,37,13.18,scattered clouds


In [8]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                236
City                   236
Country                235
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Kapoeta,SS,4.7717,33.5903,81.36,37,20,10.98,few clouds
7,7,Quisqueya,DO,18.5554,-69.4081,89.49,53,53,12.84,broken clouds
8,8,Harper,LR,4.3750,-7.7169,79.02,82,100,5.08,overcast clouds
12,12,Tuban,ID,-6.8976,112.0649,78.03,86,100,5.21,overcast clouds
17,17,Kapaa,US,22.0752,-159.3190,76.98,85,20,8.05,few clouds


In [11]:
clean_preferred_cities_df.count()

City_ID                235
City                   235
Country                235
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapoeta,SS,81.36,few clouds,4.7717,33.5903,
7,Quisqueya,DO,89.49,broken clouds,18.5554,-69.4081,
8,Harper,LR,79.02,overcast clouds,4.3750,-7.7169,
12,Tuban,ID,78.03,overcast clouds,-6.8976,112.0649,
17,Kapaa,US,76.98,few clouds,22.0752,-159.3190,
19,Bethel,US,83.28,clear sky,41.3712,-73.4140,
20,Rikitea,PF,75.69,overcast clouds,-23.1203,-134.9692,
24,Fort Dodge,US,75.02,scattered clouds,42.4975,-94.1680,
25,Kuah,MY,80.58,very heavy rain,6.3265,99.8432,
36,Pedernales,DO,87.53,scattered clouds,18.0384,-71.7440,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapoeta,SS,81.36,few clouds,4.7717,33.5903,Eastgate Hotel
7,Quisqueya,DO,89.49,broken clouds,18.5554,-69.4081,Pensión doña rosa
8,Harper,LR,79.02,overcast clouds,4.3750,-7.7169,Screensaver
12,Tuban,ID,78.03,overcast clouds,-6.8976,112.0649,Hotel Mustika
17,Kapaa,US,76.98,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
694,Meulaboh,ID,75.70,overcast clouds,4.1363,96.1285,Kost Pria
697,Ughelli,NG,75.49,moderate rain,5.4899,6.0074,Prestige Hotel
699,Luwuk,ID,76.87,overcast clouds,-0.9516,122.7875,Hotel Santika Luwuk
705,Cartagena,CO,85.62,broken clouds,10.3997,-75.5144,Hotel Colombia Real - Cartagena


In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df["Hotel Name"]==""]
#### help with code from Shruti Ramana in the ask-the-class slack channel!

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
60,Albany,US,78.64,few clouds,42.6001,-73.9662,
127,Amapa,BR,84.13,overcast clouds,1.0000,-52.0000,
214,Urubicha,BO,88.97,broken clouds,-15.3833,-62.9500,
239,Ligayan,PH,83.05,broken clouds,4.6533,119.4706,
241,Wattegama,LK,75.54,overcast clouds,6.7989,81.4808,
269,Kaniama,CD,75.72,scattered clouds,-7.5667,24.1833,
314,Yaan,NG,79.29,light rain,7.3833,8.5667,
326,Avera,US,83.55,broken clouds,33.1940,-82.5271,
340,Lodja,CD,76.24,overcast clouds,-3.4833,23.4333,
349,Guajara,BR,82.04,overcast clouds,-2.9667,-57.6667,


In [25]:
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapoeta,SS,81.36,few clouds,4.7717,33.5903,Eastgate Hotel
7,Quisqueya,DO,89.49,broken clouds,18.5554,-69.4081,Pensión doña rosa
8,Harper,LR,79.02,overcast clouds,4.3750,-7.7169,Screensaver
12,Tuban,ID,78.03,overcast clouds,-6.8976,112.0649,Hotel Mustika
17,Kapaa,US,76.98,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
694,Meulaboh,ID,75.70,overcast clouds,4.1363,96.1285,Kost Pria
697,Ughelli,NG,75.49,moderate rain,5.4899,6.0074,Prestige Hotel
699,Luwuk,ID,76.87,overcast clouds,-0.9516,122.7875,Hotel Santika Luwuk
705,Cartagena,CO,85.62,broken clouds,10.3997,-75.5144,Hotel Colombia Real - Cartagena


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/meganmcmahon/Desktop/Bootcamp/WeatherPy Mod 6/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [43]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))